# Import Libraries

In [58]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, HalvingGridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

# Load the dataset

In [59]:
df = pd.read_csv("youtube_final_csv.csv")
df.sample(10)

,video_id,title,topic,category_id,published_at,duration_seconds,video_definition,captions_flag,channel_id,channel_title,subscriber_count,total_videos,channel_age_days,views,likes,comments
38270,YcZlG14iVzE,He ABUSED a Robot For Views😂,Lifestyle (sociology),20,2025-10-03T01:45:02Z,1774,hd,False,UCiFvJHlhGHgHufH8Qxt0m0w,RICHLEV,1460000,3266,4376,225518,5120,853
4670,K21AnKC39SY,विजय सिहाग सर Rajasthan Geography Book Review ...,Knowledge,27,2025-09-14T04:30:06Z,533,hd,False,UCQGYU0IhyfXpg66jD0G2WfQ,First Book,12200,678,424,5293,54,14
19077,yHsF-FsJDE4,"Gold at $4,000 is a Terrifying Sign of What's ...",Society,24,2025-10-10T13:02:01Z,639,hd,False,UC4fg8o6oUkkZDLaC6eAZKwQ,Heresy Financial,394000,1911,2307,185991,5383,701
33719,XMJ4WCVNFSo,2014+ HARLEY DAVIDSON TOURING CHROME FORK SLID...,Lifestyle (sociology),2,2017-11-19T00:18:48Z,457,hd,False,UCTGMw_t8lbEyBGUHYVI0t3Q,SIK Baggers,125000,297,6918,137993,1013,116
39794,30BrCz0KPNg,SELF IMPROVEMENT | BNI BUSINESS CONCLAVE SURAT...,Business,27,2018-09-05T13:19:20Z,583,hd,False,UCwrzhaGy4oAHu8clPCVol3g,SONU SHARMA,12700000,524,6974,4470264,123955,1523
41893,daPW75KjiBo,Premat Kathyavar Pass| Pranit More | Marathi S...,Entertainment,23,2025-09-28T13:30:39Z,4600,hd,False,UCkGqRVFJzxFNypcbJara9uw,Pranit More Marathi,760000,472,599,2025918,43141,581
20172,PQEupVsl0HU,Mafia The Old Country is Everything Wrong With...,Action-adventure game,20,2025-08-11T17:17:27Z,1366,hd,False,UCebx-TT_kPognZx1nIfnb_g,Griffin Gaming,214000,407,4388,28591,1222,777
32634,iQCVVpVIcYc,Союзкиножурнал № 79: 25-я годовщина Октября (1...,Entertainment,1,2016-11-16T15:26:34Z,612,sd,False,UCRrRj7-bHZKjhAx5w8j65TQ,Классика советского кино (официальный канал),456000,761,4140,3287,52,5
35341,l99xSBnr_Wo,Forest Waterfall Nature Sounds - Tumbling Wate...,Lifestyle (sociology),24,2016-08-28T19:59:57Z,3724,hd,False,UC4UWbQ23IbWwU-XG7XIWxvQ,johnnielawson,453000,409,5808,36139,281,43
6118,sDMp2NecyCg,Amazon and TCS Fired 1 Lakh employees.. Why?? ...,Business,22,2025-11-10T05:48:01Z,636,hd,False,UC04CtcSH5hSM6YuRtzvFvzw,Manohar Batra - College to Corporate,97500,656,5121,6616,269,38


In [60]:
df['category_id'].value_counts()

category_id
27    10581
22     9747
26     7102
24     5812
28     3962
19     3520
20     1478
10     1347
1      1258
2      1232
17     1179
15      971
25      845
23      838
29      288
Name: count, dtype: int64

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50160 entries, 0 to 50159
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   video_id          50160 non-null  object
 1   title             50160 non-null  object
 2   topic             50160 non-null  object
 3   category_id       50160 non-null  int64 
 4   published_at      50160 non-null  object
 5   duration_seconds  50160 non-null  int64 
 6   video_definition  50160 non-null  object
 7   captions_flag     50160 non-null  bool  
 8   channel_id        50160 non-null  object
 9   channel_title     50160 non-null  object
 10  subscriber_count  50160 non-null  int64 
 11  total_videos      50160 non-null  int64 
 12  channel_age_days  50160 non-null  int64 
 13  views             50160 non-null  int64 
 14  likes             50160 non-null  int64 
 15  comments          50160 non-null  int64 
dtypes: bool(1), int64(8), object(7)
memory usage: 5.8+ MB


In [62]:
df.describe()

,category_id,duration_seconds,subscriber_count,total_videos,channel_age_days,views,likes,comments
count,50160.000000,50160.000000,5.016000e+04,50160.000000,50160.000000,5.016000e+04,5.016000e+04,50160.000000
mean,22.571531,2422.810825,2.977674e+06,5172.923405,3236.794597,2.709607e+06,3.010090e+04,1119.361643
std,6.164992,6201.971156,1.061033e+07,28809.301109,1826.909349,3.278482e+07,1.861561e+05,6548.073127
min,1.000000,240.000000,0.000000e+00,1.000000,3.000000,1.000000e+03,0.000000e+00,0.000000
25%,22.000000,630.000000,5.350000e+04,213.000000,1809.000000,2.033125e+04,4.250000e+02,29.000000
50%,24.000000,1199.000000,3.230000e+05,565.000000,3121.000000,1.385370e+05,2.630000e+03,141.000000
75%,27.000000,1990.000000,1.580000e+06,1505.000000,4581.000000,8.132555e+05,1.387225e+04,596.000000
max,29.000000,367231.000000,4.500000e+08,643411.000000,7446.000000,3.580474e+09,1.299725e+07,550394.000000


# Data Cleaning

In [63]:
df.isnull().sum()

video_id            0
title               0
topic               0
category_id         0
published_at        0
duration_seconds    0
video_definition    0
captions_flag       0
channel_id          0
channel_title       0
subscriber_count    0
total_videos        0
channel_age_days    0
views               0
likes               0
comments            0
dtype: int64

In [64]:
df.duplicated().sum()

np.int64(90)

In [65]:
# Date conversion
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')

# drop duplicates
df = df.drop_duplicates(subset=['video_id'])

# Basic filters
df = df[df['views'] > 0]
df = df[df['likes'] <= df['views']]

# Convert categorical
df['captions_flag'] = df['captions_flag'].astype(str)
df['video_definition'] = df['video_definition'].astype(str)
df['category_id'] = df['category_id'].astype(str)

# Duration cleaning
df['duration_seconds'] = df['duration_seconds'].clip(5, 7200)
df['duration_minutes'] = df['duration_seconds'] / 60

# Feature Engineering

In [66]:
df['engagement_rate_log'] = (np.log1p(df['likes']) + np.log1p(df['comments'])) / np.log1p(df['views'])
target = 'engagement_rate_log'

In [67]:
# Log-transform subscriber count
df['subscriber_count_log'] = np.log1p(df['subscriber_count'])

In [68]:
# Extract cyclical time features
df['publish_hour'] = df['published_at'].dt.hour
df['publish_dayofweek'] = df['published_at'].dt.dayofweek

df['publish_hour_sin'] = np.sin(2*np.pi*df['publish_hour']/24)
df['publish_hour_cos'] = np.cos(2*np.pi*df['publish_hour']/24)
df['publish_dow_sin'] = np.sin(2*np.pi*df['publish_dayofweek']/7)
df['publish_dow_cos'] = np.cos(2*np.pi*df['publish_dayofweek']/7)

In [69]:
# Additional features
df['is_hd'] = (df['video_definition'] == "hd").astype(int)

df['title_length'] = df['title'].str.len()
df['title_words'] = df['title'].str.split().str.len()
df['title_exclamation'] = df['title'].str.count('!')
df['title_question'] = df['title'].str.count('\\?')

In [70]:
# Drop unused columns
drop_cols = [
    'video_id','topic','title','published_at','channel_title',
    'duration_seconds','views','likes','comments','subscriber_count',
    'publish_hour','publish_dayofweek','video_definition'
]
df.drop(columns=drop_cols, inplace=True, errors="ignore")

# GROUP SPLIT

In [82]:
import joblib

categorical_features = ['category_id', 'captions_flag']
numeric_features = [
    'total_videos','channel_age_days','title_length','title_words',
    'title_exclamation','title_question',
    'duration_minutes','publish_hour_sin','publish_hour_cos',
    'publish_dow_sin','publish_dow_cos',
    'subscriber_count_log','is_hd'
]

all_features = categorical_features + numeric_features

X = df[all_features].copy()
y = df[target].copy()
groups = df['channel_id']

gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()


train_means = y_train.groupby(X_train['category_id']).mean()

X_train['category_te'] = X_train['category_id'].map(train_means)
X_test['category_te']  = X_test['category_id'].map(train_means)

numeric_features.append('category_te')

for col in categorical_features:
    cat_map = {cat: i for i, cat in enumerate(df[col].unique())}
    X_train[col] = X_train[col].map(cat_map).fillna(-1).astype(int)
    X_test[col]  = X_test[col].map(cat_map).fillna(-1).astype(int)
    
def robust_clip(t, v):
    q1, q3 = t.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    return t.clip(lo, hi), v.clip(lo, hi)

for col in ['total_videos', 'channel_age_days']:
    X_train[col], X_test[col] = robust_clip(X_train[col], X_test[col])


In [83]:
cat_maps = {}  # store all mappings

for col in categorical_features:
    cat_map = {cat: i for i, cat in enumerate(df[col].unique())}
    cat_maps[col] = cat_map  # save it
    
    X_train[col] = X_train[col].map(cat_map).fillna(-1).astype(int)
    X_test[col]  = X_test[col].map(cat_map).fillna(-1).astype(int)

# Save the mappings for Streamlit
joblib.dump(cat_maps, "categorical_mappings.pkl")


['categorical_mappings.pkl']

In [72]:
base_model = GradientBoostingRegressor(random_state=42)

param_grid = {
    "learning_rate": [0.03, 0.05],
    "n_estimators": [200, 300],
    "max_depth": [3, 4, 5],
    "min_samples_split": [2, 10],
    "min_samples_leaf": [1, 4],
    "subsample": [0.8, 1.0]
}

search = HalvingGridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    factor=2,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1
)

search.fit(X_train, y_train)
print("BEST PARAMS:", search.best_params_)

# ============================================================
# 11) FINAL MODEL
# ============================================================
final_model = GradientBoostingRegressor(
    **search.best_params_,
    random_state=42
)

final_model.fit(X_train, y_train)

n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 616
max_resources_: 39475
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 96
n_resources: 616
Fitting 3 folds for each of 96 candidates, totalling 288 fits
----------
iter: 1
n_candidates: 48
n_resources: 1232
Fitting 3 folds for each of 48 candidates, totalling 144 fits
----------
iter: 2
n_candidates: 24
n_resources: 2464
Fitting 3 folds for each of 24 candidates, totalling 72 fits
----------
iter: 3
n_candidates: 12
n_resources: 4928
Fitting 3 folds for each of 12 candidates, totalling 36 fits
----------
iter: 4
n_candidates: 6
n_resources: 9856
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 5
n_candidates: 3
n_resources: 19712
Fitting 3 folds for each of 3 candidates, totalling 9 fits
----------
iter: 6
n_candidates: 2
n_resources: 39424
Fitting 3 folds for each of 2 candidates, totalling 6 fits
BEST PARAMS: {'learning_rate': 0.03, 'max_depth': 4,

,loss,'squared_error'
,learning_rate,0.03
,n_estimators,200
,subsample,0.8
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_depth,4
,min_impurity_decrease,0.0
,init,None


In [75]:
y_pred = final_model.predict(X_test)
y_train_pred = final_model.predict(X_train)

# ============================================================
# 13) EVALUATION
# ============================================================
print("\n====== TEST PERFORMANCE ======")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2:", r2_score(y_test, y_pred))

print("\n====== TRAIN PERFORMANCE ======")
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Train R2:", r2_score(y_train, y_train_pred))


====== TEST PERFORMANCE ======
MAE: 0.15497050132639917
RMSE: 0.2117415667307127
R2: 0.25861998312955525

====== TRAIN PERFORMANCE ======
Train RMSE: 0.1952141081690276
Train R2: 0.35534870536460705


In [77]:
import joblib

joblib.dump(final_model, "engagement_xgb_model.pkl")
joblib.dump(train_means, "category_te_mapping.pkl")
joblib.dump(numeric_features, "numeric_features.pkl")
joblib.dump(categorical_features, "categorical_features.pkl")


['categorical_features.pkl']

In [78]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# ---------------------------
# Load Model + Encoders
# ---------------------------
model = joblib.load("engagement_xgb_model.pkl")
category_te_map = joblib.load("category_te_mapping.pkl")
numeric_features = joblib.load("numeric_features.pkl")
categorical_features = joblib.load("categorical_features.pkl")

st.title("🎯 YouTube Engagement Rate Predictor")
st.write("Enter video & channel details below to predict expected engagement rate.")

# --------------------------------
# Category Mapping
# --------------------------------
category_map = {
    1: "Film & Animation",
    2: "Autos & Vehicles",
    10: "Music",
    15: "Pets & Animals",
    17: "Sports",
    20: "Gaming",
    22: "People & Blogs",
    23: "Comedy",
    24: "Entertainment",
    25: "News & Politics",
    26: "How-to & Style",
    27: "Education",
    28: "Science & Technology",
    29: "Nonprofits & Activism"
}

# Invert map → name → id
category_name_to_id = {v: k for k, v in category_map.items()}

# ---------------------------
# User Inputs
# ---------------------------

# Show category names, not IDs
category_name = st.selectbox("Category", options=list(category_name_to_id.keys()))

# Convert back to numeric ID for model
category_id = category_name_to_id[category_name]

captions_flag = st.selectbox("Captions Flag", ["True", "False"])

total_videos = st.number_input("Total Videos on Channel", min_value=1, max_value=100000, value=100)
channel_age_days = st.number_input("Channel Age (days)", min_value=1, value=1000)

duration_minutes = st.slider("Video Duration (minutes)", 0.1, 120.0, 5.0)

publish_hour = st.slider("Publish Hour", 0, 23, 12)
publish_dayofweek = st.slider("Day of Week (0=Mon ... 6=Sun)", 0, 6, 3)

subscriber_count = st.number_input("Subscriber Count", min_value=0, value=10000)

is_hd = st.selectbox("Is HD?", [0, 1])

title = st.text_input("Video Title", "My Awesome Video!")

# --------------------------------
# Feature Engineering
# --------------------------------

title_length = len(title)
title_words = len(title.split())
title_exclamation = title.count("!")
title_question = title.count("?")

subscriber_count_log = np.log1p(subscriber_count)

publish_hour_sin = np.sin(2 * np.pi * publish_hour / 24)
publish_hour_cos = np.cos(2 * np.pi * publish_hour / 24)
publish_dow_sin = np.sin(2 * np.pi * publish_dayofweek / 7)
publish_dow_cos = np.cos(2 * np.pi * publish_dayofweek / 7)

# Target encoding value from training map
category_te = category_te_map.get(category_id, category_te_map.mean())

# --------------------------------
# Build Input Data
# --------------------------------
input_data = {
    "category_id": category_id,
    "captions_flag": captions_flag,
    "total_videos": total_videos,
    "channel_age_days": channel_age_days,
    "title_length": title_length,
    "title_words": title_words,
    "title_exclamation": title_exclamation,
    "title_question": title_question,
    "duration_minutes": duration_minutes,
    "publish_hour_sin": publish_hour_sin,
    "publish_hour_cos": publish_hour_cos,
    "publish_dow_sin": publish_dow_sin,
    "publish_dow_cos": publish_dow_cos,
    "subscriber_count_log": subscriber_count_log,
    "is_hd": is_hd,
    "category_te": category_te
}

input_df = pd.DataFrame([input_data])

# Convert categorical types (same as training)
input_df["category_id"] = input_df["category_id"].astype("category")
input_df["captions_flag"] = input_df["captions_flag"].astype("category")

# -----------------------------
# Prediction
# -----------------------------
if st.button("Predict Engagement Rate"):
    pred = model.predict(input_df)[0]
    engagement_rate = np.exp(pred) - 1  

    st.success(f"📈 **Predicted Engagement Rate:** {engagement_rate:.4f}")


2025-11-16 22:30:15.728 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.808 
  command:

    streamlit run /home/student/project1/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-16 22:30:15.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 22:30:15.811 Thread 'M